In [103]:
hexa = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", 'a', 'b', 'c', 'd','e','f']

def completarNumeroBits(s, numeroBits):
    while len(s) < numeroBits:
        s = "0" + s
    return s

def numberToBase(n, b):
    if n == 0:
        return [0]
    digits = []
    while n:
        digits.append(int(n % b))
        n //= b
    return digits[::-1]

def convertToPlainBase(n, b, nBits):
    res = numberToBase(n, b)
    s = ""
    for i in res:
        s += str(i)
    s = completarNumeroBits(s, nBits)
    return s

def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = -val + (1 << bits)        # compute negative value
    return val                         # return positive value as is

class Instruccion:
    def __init__ (self, instruccion):
        self.instruccion = instruccion
    def validar (self):
        if len(self.instruccion != 64):
            return false
        else:
            for i in self.instruccion:
                if i != "0" & i != "1":
                    return false
        return true
    def printInHexadecimal(self):
        decimal = int(self.instruccion, 2)
        res = numberToBase (decimal, 16)
        print (f"[{len(res)}]", end=" ")
        for i in res:
            print(hexa[i], end=" ")
        print()
    def print(self):
        print(self.instruccion)
class Comparacion: 
    def __init__ (self, instruccion):
        self.comparacion = instruccion
    def extrusion(self):
        return self.comparacion

class Procesador16:
    def __init__ (self):
        self.procesador = "procesador-16"
    def IN (self, r1):
        return Instruccion("0000" + r1 + "000000000000010000000000000000000000000000000000000")
    def ADDI (self, r3, r1, r2):
        return Instruccion("0100" + r3 + r1 + r2 + "110000000000000000000000000000000000000")
    def SUB (self, r3, r1, r2):
        return Instruccion("0101" + r3 + r1 + r2 + "110000000000000000000000000000000000000")
    def CMPLT_JL (self, r1, r2, i):
        return Instruccion("1000000000" + r1 + r2 + "0000000000000000000001" + i + "0")
    def OUT (self, r1):
        return Instruccion("0000000000000000" + r1 + "000000000000000000000000000000000000001")
    def RESETREGFILE(self):
        return Instruccion("0000000000000000000000001000000000000000000000000000000000000")
    def XOR (self, r3, r1, r2):
        return Instruccion("0010" + r3 + r1 + r2 + "110000000000000000000000000000000000000")
    def ANDI (self, r3, r1, r2):
        return Instruccion("0000" + r3 + r1 + r2 + "110000000000000000000000000000000000000")
    def NOT (self, r2, r1):
        return Instruccion("0011" + r2 + r1 + "000000" + "110000000000000000000000000000000000000")
    def SHA (self, r3, r1, r2):
        return Instruccion("0110" + r3 + r1 + r2 + "110000000000000000000000000000000000000")
    def SHL (self, r3, r1, r2):
        return Instruccion("0111" + r3 + r1 + r2 + "110000000000000000000000000000000000000")
    def IF (self, comp: Comparacion, linea):
        return Instruccion(comp.extrusion() + "10" +  convertToPlainBase(linea, 2, 16) + "0")
    def IFNOT (self, comp: Comparacion, linea):
        return Instruccion(comp.extrusion() + "01" +  convertToPlainBase(linea, 2, 16) + "0")
    def JA (self, linea):
        return Instruccion("00000000000000000000000000000000000000000011" + convertToPlainBase(linea, 2, 16) + "0")

    def CMPLT (self, r1, r2):
        return Comparacion("1000000000" + r1 + r2 + "00000000000000000000")
    def CMPLTE (self, r1, r2):
        return Comparacion("1001000000" + r1 + r2 + "00000000000000000000")
    def CMPLTU (self, r1, r2):
        return Comparacion("1100000000" + r1 + r2 + "00000000000000000000")
    def CMPLTEU (self, r1, r2):
        return Comparacion("1101000000" + r1 + r2 + "00000000000000000000")
    def EQ (self, r1, r2):
        self.comparacion = "1011000000" + r1 + r2 + "00000000000000000000"
        return Comparacion("1011000000" + r1 + r2 + "00000000000000000000")
    


In [104]:
procesador = Procesador16()

In [106]:
# Programa para calcular el MCD en mi procesador
R0 = "000000"
R1 = "000001"
procesador.RESETREGFILE().printInHexadecimal() # 0
procesador.IN(R0).printInHexadecimal() # 1
procesador.IN (R1).printInHexadecimal() # 2
procesador.IF (procesador.EQ(R0, R1), 9).printInHexadecimal() ## Salta a la línea 9 si A == B # 3
procesador.IFNOT (procesador.CMPLTE (R0, R1), 7).printInHexadecimal() ## Salta a la línea 7 si NO A <= B (A > B) #4
procesador.SUB (R1, R1, R0).printInHexadecimal() #5
procesador.JA (3).printInHexadecimal() # Salta a la línea 3 SIEMPRE #6
procesador.SUB (R0, R0, R1).printInHexadecimal() #7
procesador.JA (3).printInHexadecimal() # Salta a la línea 3 SIEMPRE #8
procesador.OUT (R0).printInHexadecimal() #9
procesador.JA(0).printInHexadecimal() # Ejecuta el programa en bucle infinito. #10

[10] 1 0 0 0 0 0 0 0 0 0 
[10] 2 0 0 0 0 0 0 0 0 0 
[13] 8 0 0 2 0 0 0 0 0 0 0 0 0 
[16] 1 6 0 0 0 0 8 0 0 0 0 4 0 0 1 2 
[16] 1 2 0 0 0 0 8 0 0 0 0 2 0 0 0 e 
[15] a 0 8 2 0 6 0 0 0 0 0 0 0 0 0 
[5] 6 0 0 0 6 
[15] a 0 0 0 0 e 0 0 0 0 0 0 0 0 0 
[5] 6 0 0 0 6 
[1] 1 
[5] 6 0 0 0 0 
